In [3]:
import numpy as np # Biblioteca para operações numéricas e álgebra linear
import pandas as pd # Biblioteca para manipulação de dados em DataFrames
import joblib # Biblioteca para salvar e carregar modelos treinados


import os
# Percorre os arquivos dentro do diretório '/kaggle/input' e imprime seus caminhos
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [4]:
# Carrega os dados de treino e teste do conjunto Titanic
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
train_data.head() # Exibe as primeiras linhas do conjunto de treino
test_ids = test_data["PassengerId"] # Armazena os IDs dos passageiros do conjunto de teste para referência futura

# Função para limpar os dados, removendo colunas irrelevantes e lidando com valores ausentes
def clean(train_data):
    # Remove colunas que não não usarei para esse projeto
    train_data = train_data.drop(["Ticket", "Cabin", "Name", "PassengerId"], axis=1)

    # Lista de colunas numéricas que podem conter valores ausentes
    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        # Preenche valores ausentes com a mediana da coluna
        train_data[col].fillna(train_data[col].median())

    # Preenche valores ausentes na coluna categórica "Embarked" com 'U' (desconhecido)
    train_data.Embarked.fillna("U") 
    return train_data

# Aplica a limpeza de dados nos conjuntos de treino e teste
train_data = clean(train_data)
test_data = clean(test_data)
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [5]:
from sklearn import preprocessing # Importa o módulo de pré-processamento do Scikit-learn
le = preprocessing.LabelEncoder()

# Colunas categóricas convertidas em valores numéricos
cols = ["Sex", "Embarked"]

for col in cols:
    train_data[col] = le.fit_transform(train_data[col]) # Ajusta o LabelEncoder e transforma os dados de treino
    test_data[col] = le.transform(test_data[col]) # Aplica a mesma transformação ao conjunto de teste
    print(le.classes_) # Exibe as classes identificadas para referência

train_data.head(5) # Exibe as primeiras 5 linhas do conjunto de treino após pré-processamento


['female' 'male']
['C' 'Q' 'S' nan]


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [6]:
def check_missing_values(data, dataset_name): # Função para verificar valores ausentes em um DataFrame
    missing_values = data.isnull().sum()  # Soma de valores ausentes por coluna
    total_missing = missing_values.sum()  # Soma total de valores ausentes
    if total_missing > 0:
        print(f"Conjunto {dataset_name} possui {total_missing} valores ausentes.")
        print("Valores ausentes por coluna:")
        print(missing_values[missing_values > 0])  # Exibir apenas colunas com valores ausentes
    else:
        print(f"Conjunto {dataset_name} não possui valores ausentes.")

check_missing_values(train_data, "train_data")
check_missing_values(test_data, "test_data")


Conjunto train_data possui 177 valores ausentes.
Valores ausentes por coluna:
Age    177
dtype: int64
Conjunto test_data possui 87 valores ausentes.
Valores ausentes por coluna:
Age     86
Fare     1
dtype: int64


In [7]:
# Calcular a média de "Fare" no test_data
fare_mean = test_data["Fare"].mean()

# Imputar valores ausentes em "Fare" com a média
test_data["Fare"] = test_data["Fare"].fillna(fare_mean)
# Calcular a média de "Age" nos conjuntos de dados
age_mean_train = train_data["Age"].mean()
age_mean_test = test_data["Age"].mean()

# Imputar valores ausentes em "Age" com a média
train_data["Age"] = train_data["Age"].fillna(age_mean_train)
test_data["Age"] = test_data["Age"].fillna(age_mean_test)

# Verificar se os valores ausentes foram tratados
print("Valores ausentes em train_data:")
print(train_data.isnull().sum())

print("\nValores ausentes em test_data:")
print(test_data.isnull().sum())


Valores ausentes em train_data:
Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

Valores ausentes em test_data:
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Define as variáveis dependente e independentes
y = train_data["Survived"]
X = train_data.drop("Survived", axis=1)

# Divide os dados em conjunto de treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Treinar o modelo de Regressão Logística
clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)
# Salvar o modelo treinado
joblib.dump(clf, 'titanic_model.pkl')


['titanic_model.pkl']

In [10]:
# Fazer previsões no conjunto de validação
predictions = clf.predict(X_val)
from sklearn.metrics import accuracy_score
# Importar e calcular a acurácia do modelo
accuracy_score(y_val, predictions)

0.8100558659217877

In [11]:
submission_preds = clf.predict(test_data)

# Agora, ambos têm o mesmo comprimento
df = pd.DataFrame({
    "PassengerId": test_ids.values,
    "Survived": submission_preds
})
# Salvar o arquivo de submissão para o Kaggle
df.to_csv("submission.csv", index=False)


import os
print(os.getcwd())  # Mostra o diretório atual
print("Arquivo salvo como submission.csv")



/kaggle/working
Arquivo salvo como submission.csv
